# DEPRESSION HELPER

## Scraper PRAW Portion

In [3]:
import praw

flairs = ['flair_name:"PROVIDING SUPPORT"','flair_name:"PROVIDING ADVICE"','flair_name:"MOTIVATION"','flair_name:"INSPIRATION"']

reddit = praw.Reddit(client_id='2n-JkPtToluQ1A',
                     client_secret='2GsEl4UykIXidjZUTtnHFItoR1Y',
                     user_agent='cs410')
print(reddit.read_only)
s = [0,0,0,0]
f = open('content.txt', "wb")
for i in range(len(flairs)):
    print('Adding', flairs[i], 'to content.txt.')
    for submission in reddit.subreddit('depression_help').search(flairs[i], syntax='cloudsearch', limit=1000):
        cur = ""
        title = submission.title
        title = title.strip('\n')
        #title = title.encode("utf-8")
        content = submission.selftext.strip()
        #content = content.encode("utf-8")
        cur = title + content + '/n'
        #cur += content
        f.write(cur.encode("utf-8"))
        s[i]+=1
print('Documents per filter/flair:',s) 
f.close() 

True
Adding flair_name:"PROVIDING SUPPORT" to content.txt.
Adding flair_name:"PROVIDING ADVICE" to content.txt.
Adding flair_name:"MOTIVATION" to content.txt.
Adding flair_name:"INSPIRATION" to content.txt.
Documents per filter/flair: [78, 67, 71, 60]


In [4]:
with open('content.txt', "r", encoding="utf8") as f:
    lines = f.readlines()
f.close()

with open('data/helpDocs.txt', "w", encoding="utf8") as f:
    for line in lines:
        if len(line) > 200:
            f.write(line)    
f.close()            

In [5]:
print('Stripping and filtering.')
uniqlines = set(open('data/helpDocs.txt', encoding="utf8").readlines())
bar = open('data/helpDocs.txt', 'w',encoding="utf8").writelines(set(uniqlines))

fn = open('data/helpDocs.txt', "r",encoding="utf8")
lines = fn.readlines()

avg = sum([len(line.strip('\n')) for line in lines]) / len(lines)
print("Average length of a post is", avg)
print("\nAll documents added to data/helpDocs.txt!")
    

Stripping and filtering.
Average length of a post is 553.599358974359

All documents added to data/helpDocs.txt!


### PLSA Portion

In [2]:
import numpy as np
import math
import sys,os
from collections import Counter 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def normalize(input_matrix):
    row_sums = input_matrix.sum(axis=1)
    assert (np.count_nonzero(row_sums)==np.shape(row_sums)[0]) # no row should sum to zero
    new_matrix = input_matrix / row_sums[:, np.newaxis]
    return new_matrix

       
class Corpus(object):
    def __init__(self, document):
        self.documents = []
        self.vocabulary = []
        self.likelihoods = []
        self.document = document
        #self.documents_path = documents_path
        self.term_doc_matrix = None 
        self.document_topic_prob = None  # P(z | d)
        self.topic_word_prob = None  # P(w | z)
        self.topic_prob = None  # P(z | d, w)
        self.k = 0
        self.number_of_documents = 0
        self.vocabulary_size = 0

    def build_corpus(self,line):
        stop_words = set(stopwords.words('english')) 
        filtered = [word for word in line[1:].split() if not word.lower() in stop_words]
        self.documents.append(filtered)
        self.number_of_documents = 1

    def build_vocabulary(self):
        for i in range(0,self.number_of_documents):
            for j in range(0,len(self.documents[i])):
                if self.documents[i][j] not in self.vocabulary:
                    self.vocabulary.append(self.documents[i][j])
                    self.vocabulary_size+=1

    def build_term_doc_matrix(self):
        self.term_doc_matrix = np.zeros((self.number_of_documents,self.vocabulary_size))
        for i in range(0,self.number_of_documents):
            for j in range(0,self.vocabulary_size):
                docCount = Counter(self.documents[i])
                docDict = dict(docCount)
                if self.vocabulary[j] in docDict:
                    self.term_doc_matrix[i][j] = docDict[self.vocabulary[j]]
                else:
                    self.term_doc_matrix[i][j] = 0    


    def initialize_randomly(self, number_of_topics):
        self.k = number_of_topics
        self.document_topic_prob = np.random.rand(self.number_of_documents,number_of_topics)
        self.document_topic_prob = normalize(self.document_topic_prob)

        self.topic_word_prob = np.random.rand(number_of_topics,len(self.vocabulary))
        self.topic_word_prob = normalize(self.topic_word_prob)
        #pass    # REMOVE THIS
        

    def initialize_uniformly(self, number_of_topics):
        self.document_topic_prob = np.ones((self.number_of_documents, number_of_topics))
        self.document_topic_prob = normalize(self.document_topic_prob)

        self.topic_word_prob = np.ones((number_of_topics, len(self.vocabulary)))
        self.topic_word_prob = normalize(self.topic_word_prob)

    def initialize(self, number_of_topics, random=False):
        if random:
            self.initialize_randomly(number_of_topics)
        else:
            self.initialize_uniformly(number_of_topics)

    def expectation_step(self):
        for i in range(0,self.number_of_documents):
            for j in range(0,self.vocabulary_size):
                denom = 0
                for k in range(0,self.k):
                    self.topic_prob[i,k,j] = self.document_topic_prob[i,k]*self.topic_word_prob[k,j]
                    denom+=self.topic_prob[i,k,j]
                if denom==0:
                    for k in range(0,self.k):
                        self.topic_prob[i,k,j]=0
                else:
                    for k in range(0,self.k):
                        self.topic_prob[i,k,j]/=denom

    def maximization_step(self, number_of_topics):
        # update P(w | z)
        for k in range(0,number_of_topics):
            denom = 0
            for j in range(0,self.vocabulary_size):
                self.topic_word_prob[k][j] = 0
                for i in range(0,self.number_of_documents):
                    self.topic_word_prob[k,j]+=self.term_doc_matrix[i,j]*self.topic_prob[i,k,j]
                denom+=self.topic_word_prob[k,j]
            if denom ==0:
                for j in range(0,self.vocabulary_size):
                    self.topic_word_prob[k,j]=1/self.vocabulary_size
            else:
                for j in range(0,self.vocabulary_size):
                    self.topic_word_prob[k,j]/=denom
        self.topic_word_prob = normalize(self.topic_word_prob)
        # update P(z | d)
        for i in range(0,self.number_of_documents):
            for k in range(0,number_of_topics):
                self.document_topic_prob[i][k]=0
                denom = 0
                for j in range(0,self.vocabulary_size):
                    self.document_topic_prob[i][k]+=self.term_doc_matrix[i][j]*self.topic_prob[i,k,j]
                    denom +=self.term_doc_matrix[i][j]
                if denom==0:
                    self.document_topic_prob[i][k]=1/number_of_topics
                else:
                    self.document_topic_prob[i][k]/=denom
        self.document_topic_prob = normalize(self.document_topic_prob)

    def calculate_likelihood(self, number_of_topics):
        log_likelihood = 0
        for i in range(0,self.number_of_documents):
            for j in range(0,self.vocabulary_size):
                tmp=0
                for k in range(0,number_of_topics):
                    tmp+=self.topic_word_prob[k][j]*self.document_topic_prob[i][k]
                if tmp>0:
                    log_likelihood+=self.term_doc_matrix[i][j]*math.log(tmp)
        return log_likelihood

    def plsa(self, number_of_topics, max_iter, epsilon):
        self.build_term_doc_matrix()

        # P(z | d, w)
        self.topic_prob = np.zeros([self.number_of_documents, number_of_topics, self.vocabulary_size], dtype=np.float)

        # P(z | d) P(w | z)
        self.initialize(number_of_topics, random=True)

        current_likelihood = 0.0

        for iteration in range(max_iter):
            self.expectation_step()
            self.maximization_step(number_of_topics)
            current_likelihood = self.calculate_likelihood(number_of_topics)

def main():
    documents_path = 'data/userTranscript.txt'
    helpdocs_path = 'data/helpDocs.txt'

    # build help doc database, takes a little while to do... ~3min
    helpDocs = []
    with open(helpdocs_path,encoding="utf-8") as fp2:
        for line in fp2:
            corpus = Corpus(line)  # instantiate corpus
            corpus.build_corpus(line)
            corpus.build_vocabulary()
            number_of_topics = 20
            max_iterations = 50
            epsilon = 0.001
            corpus.plsa(number_of_topics, max_iterations, epsilon)

            idx = np.argmax(corpus.document_topic_prob)
            chosen_topic = corpus.topic_word_prob[idx]
            chosen_word_idx = chosen_topic.argsort()[-20:][::-1]
            help_topics = []
            for idx in chosen_word_idx:
                chosen_word = corpus.vocabulary[idx]
                help_topics.append(chosen_word)
            helpDocs.append(help_topics)
    print('Help doc db built')

    # save a local data structure for all help docs for future usage
    allHelpDocs = open('data/helpDocs.txt', "r", encoding="utf8")
    docs = allHelpDocs.read().splitlines()

    # now for each user transcript...
    with open(documents_path,encoding="utf8") as fp:

        # for each user transcript, recommend documents!
        for line in fp:
            corpus = Corpus(line)  # instantiate corpus
            corpus.build_corpus(line)
            corpus.build_vocabulary()
            number_of_topics = 20
            max_iterations = 50
            epsilon = 0.001
            corpus.plsa(number_of_topics, max_iterations, epsilon)

            print('***** TOPIC WORD DISTR *****')
            idx = np.argmax(corpus.document_topic_prob)
            chosen_topic = corpus.topic_word_prob[idx]
            chosen_word_idx = chosen_topic.argsort()[-3:][::-1]
            topics = []
            for idx in chosen_word_idx:
                chosen_word = corpus.vocabulary[idx]
                topics.append(chosen_word)
            print(topics)
            
            # compare the user transcript vector with every help doc vector, get top 10 documents!
            sims = []
            for helpDoc in helpDocs:
                a = set(helpDoc) 
                b = set(topics)
                c = a.intersection(b)

                #jacquard similarity: https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50
                sim = float(len(c)) / (len(a) + len(b) - len(c))
                sims.append(sim)
            sims = np.array(sims)
            # get the 5 best doc

            fiveBestDocs = sims.argsort()[-5:][::-1]
            print('***** HERES WHAT I RECOMMEND... *****')
            for idx in fiveBestDocs:
                # print(idx)
                print(docs[idx])



if __name__ == '__main__':
    main()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Konrad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Help doc db built
***** TOPIC WORD DISTR *****
['school', 'even', 'kind']
***** HERES WHAT I RECOMMEND... *****
One of the things that have really helped me over the years, and even more so these days since I have a healthier perspective of myself than I did back then, is using symbolism to represent and remind me of personal truths. 
Firstly, I know what you're feeling right now is likely very intense, and there might even be a lot of physical pressure on your body. I know the thoughts in your head might be SCREAMING so loud it's hard to think clearly.
If depression sneaks in and lays me out, I can still believe I’m a king, even if I don’t feel like one, even if I'm failing to act like it. If a person I was hoping would approve of me doesn’t? I can still say, despite the knot I might be feeling in my chest over rejection, *“You know what? That’s okay, I’m still a king!”* And you wish them well and move on, knowing nothing about who you are has changed, and more importantly, they didn’

***** TOPIC WORD DISTR *****
['want', 'fucking', 'dont']
***** HERES WHAT I RECOMMEND... *****
It might be for different reasons. We might come from different angles. But fuck, people like me.. we fucking love you, we support you, and we want the fucking best for you. *We don't want to lose you.* We've lost so, so much. Please stay here, for us. Because we need you here. We need you to need us. I know that sometimes it can seem so easy to give up.. trust me, as a person who has been here as a survivor on this end, *I get it*. 
I don't care if a small, insignificant thing like this is something millions of people do every day. It's big for me. Be OK everyone, plant something!/nIn this moment...Right now, in this moment, I feel for the first time in 7 years that I might be able to survive. I might be able to live as an adult in this world. I’ll have to keep working so fucking hard to live with my mental illness, but maybe, just maybe, I can do that. Maybe someday I will be happy and want

***** TOPIC WORD DISTR *****
['like', 'I’ve', 'people']
***** HERES WHAT I RECOMMEND... *****
Equally; when I’ve left food outside the fridge, its gone off. Even if you haven’t personally measured this phenomena; its probable that at some stage in your life you were *told* that leaving food outside means it will go off. You believe this to be real and have no immediate reason to question it. It's not contentious. We can objectively confirm this belief by comparing food in and out of a fridge and checking how quickly microbes and bacteria spread. Some beliefs can be measured; yet some, are more subjective. For instance; *forks*. In China, people believe that the most optimal way to eat noodles in through *chopsticks*. Why is this? Why when presented with a very similar food, people have different beliefs on the best way to consume it? I mean - it's not like spaghetti is massively different to noodles so why don't we largely use chopsticks to eat Spaghetti in Europe or the US? And why ar

***** TOPIC WORD DISTR *****
['difficult', 'time', 'Stuck']
***** HERES WHAT I RECOMMEND... *****
CHECK OUT J KRISHNAMURTI on What is thinking? Thinker and thought, Observer and the observed, who are you?, memory,consciousness,Thought and time are the root cause/nYou are not alone...Hello everyone,
*Disclaimer: I don’t respond to cynicism. If you need me to elaborate on something, I’m happy to do so! If you’re here to passive-aggressively insult me or this post, please value your time and do something productive with it. 😀*
I am actually looking forward to this somehow. I am scared at the same time but I need to do this. I don't want to be alone in this./nanxiety from not working for five months, would you be able to go back to work?This is part of my 365 days of gratitude to work through my depression and the spiral into suicide attempts over the last two years. 
I have motivation , but at the same time I don't. I am currently seeking professional help and it gets better bit by bit. A

...

***** TOPIC WORD DISTR *****
[',', 'Depression', 'body']
***** HERES WHAT I RECOMMEND... *****
Alexis/nLove each and everyone of y’all. Please try to have a good Monday and a happy Monday brothers and sisters/nSelf-Help Ideas for Depression Treatment | My Depression Treatment/nI struggle with not likely myself and always wanting to change myself, I recently started watching this guy because he is funny , but I feel like he understands that being alone and being depressed hurts , and means so much to me , because I feel like he understands/nTime stands still on Tuesday, January 27th 2015. I was left behind after the suicide of my first true love. I am still so lost, trying to find meaning in all of this. Much love to every single one of you./nYou're Going to Be Okay- ASMR for Panic Attacks, Anxiety, Depression, & More/nWhen Mom Is Emotionally Unstable. Seven Ways To Heal./nCommon fallacies I've noticed that abusers use to control others/nWhen I battled this for 4 years and I'm finally n

***** TOPIC WORD DISTR *****
['don’t', 'I’m', 'ever']
***** HERES WHAT I RECOMMEND... *****
Classmates will always be competitive and a lot of people may see my grades as bad but I think it's important to me to see that I did this.  I went from forcing to drop out to getting class average and in some cases above class average.  I still have a long way to go in terms of healthy coping and habits.  But I wanted to share this because I'm happy and I really hope someone out there sees my story as a sign that things can get better.  It sure is hard when all you see is downhill and there's no certainty on where you're heading.  But keep at it you guys, I'm here for you./nCognitive Distortions: A Serious DiscussionMy therapy session today was interesting. I became uncomfortably aware at how negative I am towards myself. Which is ironic, because I’ve always considered myself to be an optimist. But I’m only optimistic about other people, about the world, about the universe. I’m a pessimist on a

***** TOPIC WORD DISTR *****
['help', 'could', 'heal']
***** HERES WHAT I RECOMMEND... *****
They more also claims for heart health improvements, and other things, but they were not very conclusive, one of the most interesting piece of information is that it would seem to help for autistic symptoms.
The fight is not easy, nor is it ever in my experience, but you have people who care about you in the world, whether be over the expanse of the worldwide web or that you know in person, and to them you are important, significant, and they are always willing to help you.
I'm not looking for sympathy here. I don't need it. All I'm asking is that you stay here for me, so I can stay here for you. Being able to help you helps me. If you stay for no else and nothing else.. please.. stay for me. Because I need you to need me.
*Nor can love, approval, friendship, or a capacity for close, caring human relationships add one iota to your inherent worth. The great majority of depressed individuals are 

***** TOPIC WORD DISTR *****
['much', 'anyone', 'ready']
***** HERES WHAT I RECOMMEND... *****
*Nor can love, approval, friendship, or a capacity for close, caring human relationships add one iota to your inherent worth. The great majority of depressed individuals are in fact very much loved, but it doesn’t help one bit because self-love and self-esteem are missing.*   
* **Column 3 (Rational Response)**: You are now ready for the crucial step in mood transformation—substituting a more rational, less upsetting thought in the right-hand column. You do not try to cheer yourself up by rationalizing or saying things you do not believe are objectively valid. Instead, try to recognize the truth.  
Just wanted to drop in and say that if you need a friend to talk to, I'm here. Feel free to reach out to me today or any day! Just know that you are not alone!/nPls listen!! YOU ARE DEPRESSION, YOU ARE THE PERPETUATOR OF IT,STOP ALL MOVEMENT/nHow I came out of depression & this is exactly what I di

***** TOPIC WORD DISTR *****
['dealing', 'want', 'see']
***** HERES WHAT I RECOMMEND... *****
I always try to remind myself that nothing is certain, everything can and will change (for better or worse), which means appreciate the moment for what you have, and don't lose hope for what you want or need./nOnly you are responsible for you./nReminder, you are allowed to be emotional and own your feelingsI see people invalidating themselves so much on /r/depression and this sub. You have a right to feel how you do, however it may be. If you feel sad for no reason, that is okay! Or if you feel numb that is fine. Feel however you want. Try not to say I should feel this when x happens or I need to feel this way. 
I've heard people say that some people know when they are going to die, and you can see it in their eyes. That's how I felt after dealing with it for so long. I don't know how to explain it, but I could feel the end getting closer, and every day that went by felt like a countdown, and 